# CW-Husky glitch exploration

CW-Husky's FPGA includes a small logic analyzer which allows the glitch generation to be visualized. This can be helpful to understand how the glitch parameters influence the shape of the glitch.

The glitch signals captured in this notebook are digital FPGA internal signals; if you're interested in the actual shape of the glitch output, you won't get that from this; you'll need a good analog oscilloscope instead.

No target needs to be connected for this notebook.

If you don't have a Husky, you can still use this notebook to understand how Husky generates glitches; skip down to the "If you don't have Husky" section. Skip over all the preceding cells.

This is also a companion to test_husky.py, for when visual inspection of glitches is needed.

In [ ]:
SCOPE="OPENADC"
PLATFORM="CWHUSKY"

In [ ]:
import chipwhisperer as cw
scope = cw.scope(name='Husky')

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 10e6
scope.clock.adc_mul = 1

scope.adc.basic_mode = "rising_edge"

scope.trigger.triggers = "tio4"
scope.io.hs2 = "clkgen"

### Set up glitch:

By default, the glitch generation logic is disabled, so it needs to be explicitely turned on.

In [ ]:
scope.glitch.enabled = True
scope.glitch.clk_src = 'pll'
scope.clock.pll.update_fpga_vco(600e6)
scope.glitch.output = 'glitch_only'
scope.glitch.trigger_src = 'manual'
scope.glitch.repeat = 1

In [ ]:
assert scope.glitch.mmcm_locked

One glitch parameter which is unique to Husky (compared to CW-lite and CW-pro) is the glitch MMCM VCO frequency. This sets the internal frequency of the FPGA glitch MMCMs, and must be between 600 MHz and 1200 MHz. The higher the frequency, the finer the phase adjustment steps (which control the glitch shape).

`scope.glitch.phase_shift_steps` gives the number of phase shift steps in one clock cycle of `scope.glitch.clk_src` (which in our example here is 10 MHz). This is determined by the clock multiplier used to bring the glitch source clock to 600 MHz; that multiplier, multiplied by the constant 56, gives the number of phase shift steps per cycle. In our example, a clock multiplier of 60 is used to generate the 600 MHz VCO frequency, and 60 times 56 = 3360 steps.

If this is too confusing, just remember that the number of phase shift steps per cycle depends on:
1. the VCO frequency (higher frequency = more steps)
2. the glitch source clock frequency (higher frequency = fewer steps).

In [ ]:
scope.glitch.phase_shift_steps

Note that increasing the VCO frequency also increases the MMCM power consumption, which can actually account for the majority of the FPGA's power draw; you can actually see the FPGA temperature go up and down with the VCO frequency (with `scope.XADC.temp`). Keep this in mind for long glitching campaigns (if the FPGA starts getting too hot, red LEDs will flash, and the error condition will be noted on `scope.XADC.status`).

### Set up logic analyzer:

The logic analyzer is an optional component which can be included or excluded in the FPGA build process, so first let's make sure it's there:

In [ ]:
assert scope.LA.present, 'There is no logic analyzer in this FPGA bitfile!'

Like the glitch logic, the LA uses a fast MMCM which can draw significant power, so it's disabled by default.

In order to catch narrow glitches, we need a high oversampling factor. The LA sampling clock is derived from the target clock. In this case we're oversampling at 40x.

The target clock is 10 MHz, so the LA will be sampling at 400 MS/s. Technically this is faster than we should go since the FPGA bitfile is implemented to support a maximum sampling rate of 250 MS/s, but here in practice it works fine (YMMV if you're using Husky in extreme conditions!).

Finally we set the `capture_group` to capture the glitch signals (the LA can also be used to capture USERIO or 20-pin connector signals), and we set the capture trigger to be the glitch itself.

See `help(scope.LA)` for more on these and other capture parameters.

In [ ]:
scope.LA.enabled = True
scope.LA._warning_frequency = 401e6 # prevent warning message that we're setting the sampling clock too high
scope.LA.oversampling_factor = 40
scope.LA.downsample = 1
scope.LA.capture_group = 'glitch'
scope.LA.trigger_source = "glitch_source"
scope.LA.capture_depth = 512

In [ ]:
assert scope.LA.locked

### Single capture:

Let's first do a simple single capture.

We can pick arbitrary glitch `offset` and `width` parameters; start with the values below, then explore other values.

In [ ]:
# adjust as you wish:
scope.glitch.offset = 1000
scope.glitch.width = 1000

Whereas CW-lite and CW-pro have "coarse" and "fine" settings for each parameter, with Husky there is a single setting, which is the number of phase shift steps. This can be a positive or negative number, and it is allowed to roll over (e.g. with the default settings in this notebook, `scope.glitch.phase_shift_steps = 3360`; it's possible to set `offset` or `width` to, for example, 4000, which would be equivalent to 4000-3360 = 640).

While this is a change from CW-lite/pro that's not backwards compatible, it makes it much easier to adjust and understand `offset` and `width`.

(This change is due to architectural differences in the Xilinx FPGAs: CW-lite/pro have a Spartan6 FPGA and use DCMs to generate glitches; CW-Husky has an Artix7 FPGA and its more powerful MMCMs to generate glitches.)

In [ ]:
scope.LA.arm()
scope.glitch.manual_trigger()

After `manual_trigger()`, the collected data is ready to be read via `read_capture_data()`.

In [ ]:
raw = scope.LA.read_capture_data()
glitchout    = scope.LA.extract(raw, 0)
source       = scope.LA.extract(raw, 1)
mmcm1out     = scope.LA.extract(raw, 2)
mmcm2out     = scope.LA.extract(raw, 3)
glitchgo     = scope.LA.extract(raw, 4)
glitchenable = scope.LA.extract(raw, 6)
glitchsource = scope.LA.extract(raw, 7)

And now we plot:

In [ ]:
from bokeh.plotting import figure, show
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span, Legend, LegendItem
import numpy as np
output_notebook(INLINE)

o = figure(plot_width=1800)

xrange = range(len(source))
O1 = o.line(xrange, source + 6, line_color='black')
O2 = o.line(xrange, mmcm1out + 4, line_color='blue')
O3 = o.line(xrange, mmcm2out + 2, line_color='red')
O4 = o.line(xrange, glitchout + 0, line_color='purple', line_width=2)
O5 = o.line(xrange, glitchenable - 2, line_color='black', line_width=2)
#O6 = o.line(xrange, glitchgo - 4, line_color='green')
#O7 = o.line(xrange, glitchsource - 6, line_color='pink', line_width=2)

legend = Legend(items=[
    LegendItem(label='source clock', renderers=[O1]),
    LegendItem(label='glitch MMCM1 output (internal signal)', renderers=[O2]),
    LegendItem(label='glitch MMCM2 output (internal signal)', renderers=[O3]),
    LegendItem(label='glitch clock output', renderers=[O4]),
    LegendItem(label='glitch enable', renderers=[O5]),
    #LegendItem(label='glitch go', renderers=[O6]),
    #LegendItem(label='glitch trigger source', renderers=[O7]),
])
o.add_layout(legend)

Let's add some handy markers to help visualize how the glitch is constructed.

The glitch clock output is high when MMCM1, MMCM2, and glitch enable are all high. (Because `scope.glitch.output = "glitch_only"` in this example. You can change this setting to observe the different behaviours that are possible.)

In [ ]:
# add glitch markers:
def find_transitions(data, pattern):
    return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

transitions = [find_transitions(glitchout, [0,1])[0]+1, find_transitions(glitchout, [1,0])[0]]

for b in transitions:
    o.renderers.extend([Span(location=b, dimension='height', line_color='black', line_width=1, line_dash='dashed')])

In [ ]:
show(o)

You can explore the effect of different `scope.glitch` parameters (such as `width`, `offset`, and `output`) and re-run the above from the `manual_trigger()` call onwards.

### Interactive glitch visualization:

Now we step through many width/offset combinations so that we can interactively plot them.
We carry out STEPS * STEPS captures. STEPS can be whatever you want, but there it doesn't make sense to make STEPS greater that `scope.LA.oversampling_factor`.

In [ ]:
STEPS = scope.LA.oversampling_factor
increment = scope.glitch.phase_shift_steps // STEPS
start = 0

import numpy as np
glitchouts = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
sources    = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
mmcm1outs  = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
mmcm2outs  = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
glitchenables = np.zeros((STEPS, STEPS, scope.LA.capture_depth))
glitchgo = np.zeros((STEPS, STEPS, scope.LA.capture_depth))

from tqdm.notebook import tnrange

scope.glitch.offset = start
scope.glitch.width = start

for o in tnrange(STEPS):
    scope.glitch.width = start
    for w in range(STEPS):
        scope.LA.arm()
        scope.glitch.manual_trigger() 
        raw = scope.LA.read_capture_data()
        glitchouts[o][w]   = scope.LA.extract(raw, 0)
        sources[o][w]      = scope.LA.extract(raw, 1)
        mmcm1outs[o][w]    = scope.LA.extract(raw, 2)
        mmcm2outs[o][w]    = scope.LA.extract(raw, 3)
        glitchgo[o][w]     = scope.LA.extract(raw, 4)
        glitchenables[o][w] = scope.LA.extract(raw, 6)
        scope.glitch.width += increment
    scope.glitch.width = start
    scope.glitch.offset += increment

In [ ]:
#np.savez_compressed('data/husky_glitch_data.npz', array=np.asarray([glitchouts, sources, mmcm1outs, mmcm2outs, glitchgo, glitchenables]))

**If you don't have a Husky**, run this to load previously saved glitch waveform data generated by a Husky:

In [ ]:
# this is only needed if you don't have a Husky
try:
    type(scope) == 'chipwhisperer.capture.scopes.OpenADC.OpenADC'
except:
    from bokeh.plotting import figure, show
    from bokeh.resources import INLINE
    from bokeh.io import output_notebook
    import numpy as np
    output_notebook(INLINE)

    def find_transitions(data, pattern):
        return [i for i in range(0,len(data)) if list(data[i:i+len(pattern)])==pattern]

    alls = np.load('../data/husky_glitch_data.npz')
    [glitchouts, sources, mmcm1outs, mmcm2outs, glitchgo, glitchenables] = alls['array']

    STEPS = len(glitchouts)

In [ ]:
def update_plot(offset, width):
    S1.data_source.data['y'] = sources[offset][width] + 6
    S2.data_source.data['y'] = mmcm1outs[offset][width] + 4
    S3.data_source.data['y'] = mmcm2outs[offset][width] + 2 
    S4.data_source.data['y'] = glitchouts[offset][width] + 0
    S5.data_source.data['y'] = glitchenables[offset][width] - 2
    #S6.data_source.data['y'] = glitchgo[offset][width] - 4

    t1s = find_transitions(glitchouts[offset][width], [0,1])
    t2s = find_transitions(glitchouts[offset][width], [1,0])
    if len(t1s) == 1:
        T1.location = t1s[0]+1
    else:
        T1.location = 0

    if len(t2s) == 1:
        T2.location = t2s[0]
    else:
        T2.location = 0

    
    push_notebook()

In [ ]:
from ipywidgets import interact, Layout
from bokeh.io import push_notebook
from bokeh.models import Span, Legend, LegendItem

o = 0
w = 0

S = figure(plot_width=1800)

xrange = range(len(sources[o][w]))
S1 = S.line(xrange, sources[o][w] + 6, line_color='black')
S2 = S.line(xrange, mmcm1outs[o][w] + 4, line_color='blue')
S3 = S.line(xrange, mmcm2outs[o][w] + 2 , line_color='red')
S4 = S.line(xrange, glitchouts[o][w] + 0, line_color='purple', line_width=2)
S5 = S.line(xrange, glitchenables[o][w] - 2, line_color='green')
#S6 = S.line(xrange, glitchgo[o][w] - 4, line_color='pink', line_width=2)

t1s = find_transitions(glitchouts[o][w], [0,1])
t2s = find_transitions(glitchouts[o][w], [1,0])
if len(t1s) == 1:
    T1_location = t1s[0]+1
else:
    T1_location = 0

if len(t2s) == 1:
    T2_location = t2s[0]
else:
    T2_location = 0
        
T1 = Span(location=T1_location, dimension='height', line_color='black', line_dash='dashed', line_width=1)
T2 = Span(location=T2_location, dimension='height', line_color='black', line_dash='dashed', line_width=1)

legend = Legend(items=[
    LegendItem(label='source clock', renderers=[S1]),
    LegendItem(label='glitch MMCM1 output', renderers=[S2]),
    LegendItem(label='glitch MMCM2 output', renderers=[S3]),
    LegendItem(label='glitch clock output', renderers=[S4]),
    LegendItem(label='glitch enable', renderers=[S5]),
    #LegendItem(label='glitch go', renderers=[S6]),
])

S.add_layout(legend)
S.add_layout(T1)
S.add_layout(T2)

In [ ]:
show(S, notebook_handle=True)

In [ ]:
interact(update_plot, offset=(0, STEPS-1), width=(0, STEPS-1))

This concludes the exploratory portion of this notebook.

In the sections that follow, we show how the internal logic analyzer can be used to validate that glitches are being generated correctly.

If you're done, turn off MMCMs to allow the FPGA to cool down:

In [ ]:
scope.LA.enabled = False
scope.glitch.enabled = False

# Validation

The next cells reproduce some of the checks that are done in `test_husky.py`.

For a particular static offset/width setting, runs lots of captures and ensure there are no missing glitches and no extra glitches:

In [ ]:
scope.glitch.repeat = 2
from tqdm.notebook import tnrange
import numpy as np
oversamp = scope.LA.oversampling_factor
lens = []
overlens = []
zerolens = 0
for i in tnrange(1000):
    scope.LA.arm()
    scope.glitch.manual_trigger()
    raw = scope.LA.read_capture_data()
    glitchenable = scope.LA.extract(raw, 6)
    glitchlen = len(np.where(glitchenable > 0)[0])
    if not glitchlen:
        zerolens += 1
        continue
    lens.append(glitchlen)
    if abs(glitchlen/scope.glitch.repeat - oversamp) > oversamp / 4:
        overlens.append(glitchlen)

In [ ]:
assert zerolens == 0 and len(overlens) == 0

Look for "double glitches" bug:

Here we just look at "go" length, for double glitches that can't otherwise be seen.
This takes a while.

In [ ]:
scope.clock.pll.update_fpga_vco(1200e6)

In [ ]:
import numpy as np

In [ ]:
reps = 3
width = -3000
oversamp = 30
stepsize = 1

scope.glitch.width = 0
scope.glitch.offset = 0

scope.clock.pll.update_fpga_vco(1200e6)

margin = 1
prev_offset = 0
scope.LA.oversampling_factor = oversamp
scope.glitch.width = width
scope.glitch.repeat = 1

overs = []
badoffsets = []

from tqdm.notebook import tnrange

for r in tnrange(reps):
    offsets = []
    glitches = []
    sources = []

    # sweep offset and check that glitch offset increases by expected amount each time:
    for i, o in enumerate(range(0, scope.glitch.phase_shift_steps - stepsize, stepsize)):
        scope.glitch.offset = o
        scope.LA.arm()
        scope.glitch.manual_trigger()
        raw = scope.LA.read_capture_data()
        glitchgo  = scope.LA.extract(raw, 4)
        golen = len(np.where(glitchgo > 0)[0])
        if golen and (abs(golen - oversamp) > oversamp/4):
            print("Go width exceeds margin: %d at offset=%d" % (golen, o))
            overs.append(golen)
            badoffsets.append(o)


In [ ]:
scope.clock.pll.update_fpga_vco(600e6)

When done, turn off MMCMs to cool down:

In [ ]:
scope.LA.enabled = False
scope.glitch.enabled = False

In [ ]:
scope.XADC